In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS


llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini"
)
memory = ConversationBufferMemory(
    return_messages=True,
)

loader = UnstructuredFileLoader("./files/document.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstores = FAISS.from_documents(docs, cache_embeddings)
retriever = vectorstores.as_retriever()


In [11]:
template = ChatPromptTemplate.from_messages([
    ("system", 
     """
     You're a helpful assistant. Answer questions using only the following context.
     If you don't know the answer just say you don't know, don't make it up.
     ------
     {context}
     ------
     """),
     ("system", "Below is the content of the conversation so far."),
     MessagesPlaceholder(variable_name="history"),
     ("human", "{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]
# chain = RunnablePassthrough.assign(history_test=load_memory) | template | llm
# def invoke_chain(question):
#   result = chain.invoke({"question":question})
#   memory.save_context(
#     {"input":question},
#     {"output": result.content},
#   )
#   print(result)

# final_chain = {"context":map_chain, "question":RunnablePassthrough()} | final_template | llm
# final_chain.invoke("Where does Winston go to work?")
chain = {"history":load_memory, "context":retriever, "question":RunnablePassthrough()} | template | llm
# chain = RunnablePassthrough.assign(history=load_memory, context=retriever) | template | llm

def chain_invoke(question):
    result = chain.invoke(question)
    print(result)
    memory.save_context({"input": question}, {"output": result.content})

In [12]:
chain_invoke("Is Aaronson guilty?")

content='According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, despite having a memory that disproves their guilt. He concludes that he had never seen the photograph that disproved their guilt and that it had never existed; he had invented it.'


In [13]:
chain_invoke("What message did he write in the table?")

content='Winston traced "2+2=5" in the dust on the table.'


In [14]:
chain_invoke("Who is Julia?")

content='Julia is a character that Winston loves deeply. In the context provided, she is someone with whom Winston shares a romantic relationship, and he expresses a strong emotional connection to her, even in moments of despair and fear.'


In [15]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Is Aaronson guilty?'),
  AIMessage(content='According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, despite having a memory that disproves their guilt. He concludes that he had never seen the photograph that disproved their guilt and that it had never existed; he had invented it.'),
  HumanMessage(content='What message did he write in the table?'),
  AIMessage(content='Winston traced "2+2=5" in the dust on the table.'),
  HumanMessage(content='Who is Julia?'),
  AIMessage(content='Julia is a character that Winston loves deeply. In the context provided, she is someone with whom Winston shares a romantic relationship, and he expresses a strong emotional connection to her, even in moments of despair and fear.')]}

In [16]:
chain_invoke("What was my first question?")

content='Your first question was, "Is Aaronson guilty?"'
